## Fake News Classifier Using LSTM

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Fake News Classifier/Fake news classifer.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
###Drop Nan Values
df=df.dropna()

In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.19.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

# Dataset Preprocessing

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages.reset_index(inplace=True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1437, 1902, 1500, 2505, 3474, 3696, 2397, 4164, 468, 799],
 [4156, 2733, 2738, 4789, 4593, 4125, 1554],
 [4229, 4208, 2935, 3916],
 [4787, 2690, 2964, 26, 1116, 3883],
 [1778, 4593, 4716, 4469, 1006, 1715, 4593, 2739, 1814, 1865],
 [3083,
  1249,
  2138,
  756,
  4229,
  4323,
  1831,
  960,
  4828,
  4030,
  2985,
  2156,
  1411,
  1450,
  1554],
 [1161, 992, 560, 4921, 1993, 932, 1056, 2341, 1725, 1395, 1428],
 [3505, 4964, 4519, 4927, 2277, 3162, 4323, 1405, 1725, 1395, 1428],
 [28, 4330, 2334, 1421, 1844, 4815, 1689, 3135, 4323, 2543],
 [4554, 284, 1612, 2125, 3760, 4695, 2276, 4414],
 [4428, 51, 2407, 6, 4034, 610, 4279, 2014, 722, 2267, 886],
 [26, 2360, 3474, 4815, 4323, 2277],
 [643, 2445, 927, 4759, 2940, 4522, 1699, 1355, 4079],
 [703, 1947, 1842, 4083, 1097, 85, 2753, 1725, 1395, 1428],
 [3631, 348, 4187, 3784, 4493, 1725, 1395, 1428],
 [529, 1398, 745, 2170, 811, 679, 4816, 4556, 3685, 153],
 [4724, 2133, 2733],
 [1291, 293, 2647, 4433, 4323, 4378, 3401, 1554],
 [2263, 18

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr[1]

[4156, 2733, 2738, 4789, 4593, 4125, 1554]

### Embedding Representation

In [28]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1437 1902 1500 ...    0    0    0]
 [4156 2733 2738 ...    0    0    0]
 [4229 4208 2935 ...    0    0    0]
 ...
 [1164 1079  370 ...    0    0    0]
 [4508 2277 3340 ...    0    0    0]
 [3384 2938 3375 ...    0    0    0]]


In [29]:
embedded_docs[1]

array([4156, 2733, 2738, 4789, 4593, 4125, 1554,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [30]:
embedded_docs[0]

array([1437, 1902, 1500, 2505, 3474, 3696, 2397, 4164,  468,  799,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

# Define the model SimpleRNN

In [33]:
# Define the model
from keras.layers import Embedding, SimpleRNN, Dense
embedding_vector_features=40 ##features representation
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SimpleRNN(32))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:
len(embedded_docs),y.shape

(18285, (18285,))

# train_test_split

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00


# Performance Metrics And Accuracy

In [38]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5666942833471417

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       1.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.78      0.50      0.36      6035
weighted avg       0.75      0.57      0.41      6035

